In [1]:
import numpy as np
import sympy as sym
from sympy import symbols

In [7]:
symbols(r"\delta\theta")

\delta\theta

In [2]:
arr21 = np.zeros((2,2), dtype=sym.Mul)
arr22 = np.zeros((2,2), dtype=sym.Mul)
arr4 = np.zeros((4,4), dtype=sym.Mul)
for i in range(4):
    for j in range(4):
        arr4[i][j] = sym.symbols("a_{}{}".format(i, j))
        if i<2 and j<2:
            arr21[i][j] = sym.symbols("a_{}{}".format(i, j))
        elif i>=2 and j>=2:
            arr22[i-2][j-2] = sym.symbols("a_{}{}".format(i, j))



arr4[0][2] = 0
arr4[1][3] = 0
arr4[2][0] = 0
arr4[3][1] = 0

u_sym = sym.symbols("u")
v_sym = sym.symbols("v")

arr4[0][1] += -u_sym
arr4[2][3] += -u_sym

arr4[1][0] += v_sym
arr4[3][2] += v_sym

arr4[0][3] = u_sym
arr4[1][2] = -v_sym
arr4[2][1] = u_sym
arr4[3][0] = -v_sym

arr4 = sym.Matrix(arr4)
arr21 = sym.Matrix(arr21)
arr22 = sym.Matrix(arr22)

In [3]:
arr4

Matrix([
[    a_00, a_01 - u,        0,        u],
[a_10 + v,     a_11,       -v,        0],
[       0,        u,     a_22, a_23 - u],
[      -v,        0, a_32 + v,     a_33]])

In [47]:
display(sym.simplify(arr4.det() - arr21.det()*arr22.det()).collect([u_sym*v_sym, u_sym, v_sym]))
arr4.det()


u*v*(a_00*a_11 + a_00*a_33 - a_01*a_10 - a_01*a_32 - a_10*a_23 + a_11*a_22 + a_22*a_33 - a_23*a_32) + u*(a_00*a_11*a_32 - a_01*a_10*a_32 + a_10*a_22*a_33 - a_10*a_23*a_32) + v*(-a_00*a_11*a_23 + a_01*a_10*a_23 - a_01*a_22*a_33 + a_01*a_23*a_32)

a_00*a_11*a_22*a_33 - a_00*a_11*a_23*a_32 - a_00*a_11*a_23*v + a_00*a_11*a_32*u + a_00*a_11*u*v + a_00*a_33*u*v - a_01*a_10*a_22*a_33 + a_01*a_10*a_23*a_32 + a_01*a_10*a_23*v - a_01*a_10*a_32*u - a_01*a_10*u*v - a_01*a_22*a_33*v + a_01*a_23*a_32*v - a_01*a_32*u*v + a_10*a_22*a_33*u - a_10*a_23*a_32*u - a_10*a_23*u*v + a_11*a_22*u*v + a_22*a_33*u*v - a_23*a_32*u*v

In [3]:
H, Htet, Hphi = symbols("H theta_H phi_H", real=True)
Ms = symbols("Ms", real=True)
Mtet = symbols("theta_M1 theta_M2", real=True)
Mphi = symbols("phi_M1 phi_M2", real=True)
A, Ku2, g = symbols("A K_u2 gamma", real=True)
J = symbols("J", real=True)
Hk1, Hk2, H2k = symbols("H_k1 H_k2 H2_k", real=True)
t = symbols("t", real=True)

Ev1 = -Ms * H * (sym.sin(Htet) * sym.sin(Mtet[0]) * sym.cos(Hphi - Mphi[0]) +
                 sym.cos(Htet) * sym.cos(Mtet[0])) + \
      A * (sym.sin(Mtet[0]) ** 2) + \
      Ku2 * (sym.sin(Mtet[0]) ** 4)

Ev2 = -Ms * H * (sym.sin(Htet) * sym.sin(Mtet[1]) * sym.cos(Hphi - Mphi[1]) +
                 sym.cos(Htet) * sym.cos(Mtet[1])) + \
      A * (sym.sin(Mtet[1]) ** 2) + \
      Ku2 * (sym.sin(Mtet[1]) ** 4)

Es = t * Ev1 + t * Ev2 + \
     J * (sym.sin(Mtet[0]) * sym.sin(Mtet[1]) * sym.cos(Mphi[1] - Mphi[0]) +
          sym.cos(Mtet[0]) * sym.cos(Mtet[1]))


system_vars = [Mtet[0], Mtet[1], Mphi[0], Mphi[1]]
system_mat = np.zeros((4, 4), dtype=sym.Add)
for i in range(2):
    for j in range(4):
        system_mat[i][j] = (g / Ms) * (1 / t) * Es.diff(Mphi[i], system_vars[j])

for i in range(2, 4):
    for j in range(4):
        system_mat[i][j] = -(g / Ms) * (1 / t) * Es.diff(Mtet[i - 2], system_vars[j])

w = symbols("w", real=True)
jw = sym.I * w

for i in range(4):
    system_mat[i][i] += jw * sym.sin(Mtet[i % 2])

system_mat = sym.Matrix(system_mat)

# subs for PMA
system_mat = system_mat.subs({Hphi: 0, Mphi[0]: 0, Mphi[1]: 0})
system_mat = system_mat.subs({Ku2: Ms * Hk2 / 4})
system_mat = system_mat.subs({A: Ms * (Hk1 - Hk2) / 2})
system_mat = system_mat.subs({J: (t / 2) * Ms * (H2k - Hk1)})
system_mat.simplify()

w_eq = sym.solve(system_mat.det(), w)
f_eq = np.array(w_eq)/(2*sym.pi)
varset = (
    H, Htet,
    Ms, Mtet[0], Mtet[1],
    Hk1, Hk2, H2k,
    t,
    g
)

f_res_lamb = []
for eq in f_eq:
    f_res_lamb.append(sym.lambdify(varset, eq, 'numpy'))

In [15]:
vars_sol = symbols("v0 v1 v2 v3", complex=True)
eqs = [0, 0, 0]
for i in range(3):
    for j in range(4):
        eqs[i] += (vars_sol[j])*system_mat[i + 1, j]

v_sol = sym.solve(eqs, vars_sol)

KeyboardInterrupt: 

In [170]:



class LinearDouble:
    def __init__(self):
        

    def freq(
            self,
            H, Htet,
            Ms, Mtet1, Mtet2,
            Hk1, Hk2, H2k,
            t,
            g,
            ind=1
    ):
        return self.f_res_lamb[ind](
            H, Htet,
            Ms, Mtet1, Mtet2,
            Hk1, Hk2, H2k,
            t,
            g,
        )



c*(H*gamma*cos(theta_M1)/(w*sin(theta_M1)) - 2*H_k1*gamma*sin(theta_M1)/w + H_k1*gamma/(w*sin(theta_M1)) - 4*H_k2*gamma*sin(theta_M1)**3/w + 5*H_k2*gamma*sin(theta_M1)/w - H_k2*gamma/(w*sin(theta_M1)))